In [24]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes
!pip install -U openai
!pip install htmlrag

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/Uminchu618/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 72 (delta 3), reused 9 (delta 3), pack-reused 62 (from 1)
Receiving objects: 100% (72/72), 88.50 KiB | 22.13 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random
random.seed(0)

In [6]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [7]:
# 質問のリスト
questions = [
    "すでに構築したクラウド環境を IaC に変換する機能はありますか？",
    "大量の機械学習トレーニングデータはどこに保管すべきですか？",
    "SageMaker のトレーニングジョブが S3 にアクセスするには IAM ユーザーが必要ですか？",
    "トレーニング費用を抑える方法はありますか？",
    "Notebook Instance と SageMaker Studio はどう使い分けるべきですか？",
    "BedrockでDeepSeek-R1を使った場合、どのくらいの推論コストがかかりますか？"
]

# 回答のリスト
answers = [
    "はい。AWS CloudFormation の *Resource Import* 機能では、既存リソースをテンプレートに記述してスタックへ取り込みます。Terraform でも `terraform import` コマンドを使い、状態ファイルに既存リソースを登録できます。複雑なスタックでは、インポート後にパラメータや依存関係を手動でレビュー・修正するのが一般的です。",
    "Amazon S3 が推奨です。オブジェクトを自動で冗長化し、11 ナイン (99.999999999 %) の耐久性を提供します。データレイクや SageMaker とネイティブ統合され、ペタバイト級でもスケールを意識せずに利用できます。S3 イベントで Lambda を呼び出せば、取込直後の前処理やメタデータ登録も自動化できます。",
    "IAM ユーザーは不要です。SageMaker には **IAM 実行ロール** を付与し、そのロールに `s3:*` など必要なポリシーをアタッチします。ジョブ実行時、SageMaker はこのロールを引き受けて S3 や CloudWatch などのサービス API を呼び出します。",
    "Amazon SageMaker **Managed Spot Training** を有効にすると、オンデマンド比で最大 90 % のコスト削減が可能です。Spot 中断に備えてチェックポイントを S3 に保存する仕組みがあり、再開時に自動で引き継げます。学習時間よりコスト優先のワークロードで効果が高いベストプラクティスです。",
    "Studio Classic ノートブックは起動が Notebook Instance より 5–10 倍速く、インスタンス種別もブラウザから即変更できます。Notebook Instance はシングルユーザーで完全に分離した環境が欲しい場合や、従来の CLI/SDK 手順を重視するケースに向きます。いずれも従量課金ですが、Studio はアイドル状態の EFS ストレージ料金、Instance は EC2 実行料金の最小化がポイントです。",
    "1,000 入力トークンあたりの料金は 0.00135 USD、1,000 出力トークンあたりの料金は 0.0054 USDです。これは、他のオープンモデル（Llama 3.3 など）に対して比較的高い料金となっています。"
]


In [9]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [17]:
# システムプロンプト
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。１００文字以内で回答すること。"
responses = []
# 各質問に回答を生成
for i, q in enumerate(questions, start=1):
    print(f"### {i}. {q}")
    response = generate_output(q, system_prompt)
    print("**回答**")
    print(response)
    print("\n" + "---" + "\n")
    responses.append(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


### 1. すでに構築したクラウド環境を IaC に変換する機能はありますか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


**回答**
はい、すでに構築したクラウド環境を Infrastructure as Code (IaC) に変換する機能はあります。例えば、AWS CloudFormation、Azure Resource Manager (ARM)、Google Cloud Deployment Manager などのクラウドプロバイダーが提供するツールや、third-party のソフトウェアとして、Cloudify、Terraform、Ansible などがあります。これらのツールを使用することで、既に構築されたクラウド環境を IaC に変換し、自動化されたインフラストラクチャーの管理を実現することができます。

---

### 2. 大量の機械学習トレーニングデータはどこに保管すべきですか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

**回答**
はい、SageMaker のトレーニングジョブが S3 にアクセスするには、IAM ユーザーが必要です。SageMaker は、IAM 認証を使用して S3 にアクセスし、データを読み書きするために、IAM ユーザーが必要です。SageMaker は、IAM ユーザーに紐付けたロールを使用して、S3 にアクセスし、データを読み書きします。

---

### 4. トレーニング費用を抑える方法はありますか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


**回答**
トレーニング費用を抑える方法はいくつかあります。まず、オンライントレーニングやMOOC（Massive Open Online Course）を活用することができます。これにより、コストを大幅に削減することができます。次に、自宅で実施するトレーニングや、同僚や友人と共同で実施するトレーニングも有効です。また、企業の内部トレーニングや、政府のサポートするトレーニングプログラムも活用することができます。最後に、トレーニングの内容を効率的に設計し、必要最小限度の時間で実施することも重要です。

---

### 5. Notebook Instance と SageMaker Studio はどう使い分けるべきですか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


**回答**
Notebook InstanceとSageMaker Studioは、AWS SageMakerの2つのサービスです。Notebook Instanceは、Jupyter NotebookやApache Zeppelinなどのnotebook環境を提供し、データサイエンティストや機械学習エンジニアがコードを実行するための環境を提供します。SageMaker Studioは、データサイエンティストや機械学習エンジニアが、機械学習モデルを開発、トレーニング、デプロイするための統合的なワークフローアプリケーションです。

Notebook Instanceは、個々のnotebook環境を提供し、各々のプロジェクトやタスクに特化した環境を提供します。SageMaker Studioは、複数のnotebook環境を統合的に管理し、プロジェクトの全体像を把握することができます。したがって、Notebook Instanceは、個々のnotebook環境を提供し、SageMaker Studioは、複数のnotebook環境を統合的に管理するためのサービスです。

---

### 6. BedrockでDeepSeek-R1を使った場合、どのくらいの推論コストがかかりますか？
**回答**
BedrockのDeepSeek-R1は、推論コストを最小化するために設計された深層学習ベースの推論エンジンです。具体的には、推論コストはBedrockのDeepSeek-R1の性能や設定によって異なります。

しかし、Bedrockの公式ドキュメントによると、DeepSeek-R1は、推論コストを平均して1-10ミリ秒程度と報告されています。ただし、実際の推論コストは、データサイズ、モデルサイズ、ハードウェアの性能など、多くの要因によって影響を受けるため、実際の推論コストは異なる可能性があります。

---



In [19]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

def openai_generator(query):

        messages = [
                    {
                        "role": "user",
                        "content": query
                    }
                ]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
          "Say 4, if User Answer is full contained and equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
          "### Question: {query}\n"
          "### {answer0}: {sentence_inference}\n"
          "### {answer1}: {sentence_true}\n"
          "The rating is:\n"
      )
    template_accuracy2 = (
          "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
          "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
          "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
          "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
          "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
          "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
          "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
          "Question: {query}\n\n"
          "{answer0}: {sentence_inference}\n\n"
          "{answer1}: {sentence_true}\n\n"
          "Rating: "
      )

    score1 = openai_generator(
                template_accuracy1.format(
                      query=query,
                      answer0="User Answer",
                      answer1="Reference Answer",
                      sentence_inference=response,
                      sentence_true=reference,
                    )
                )
    try:
      score1 = int(score1)
    except:
      print("Failed")
      score1 = 0

    score2 = openai_generator(
                template_accuracy2.format(
                        query=query,
                        answer0="Reference Answer",
                        answer1="User Answer",
                        sentence_inference=reference,
                        sentence_true=response,
                    )
                  )

    try:
      score2 = int(score2)
    except:
      print("Failed")
      score2 = 0


    return (score1 + score2) / 2

In [22]:
print("| 評価番号 | スコア |")
print("|:--------:|:------:|")

# Iterate and print each row with only index and score
for i, (question, gold_answer, response) in enumerate(zip(questions, answers, responses), start=1):
    score = evaluate_answer_accuracy(question, response, gold_answer)
    print(f"| {i} | {score:.2f} |")


| 評価番号 | スコア |
|:--------:|:------:|
| 1 | 2.00 |
| 2 | 0.00 |
| 3 | 0.00 |
| 4 | 0.00 |
| 5 | 0.00 |
| 6 | 0.00 |


In [67]:
import htmlrag
import re
html=open("./Bedrock_FAQ.html").read()
simplified_html=htmlrag.clean_html(html)
block_tree, simplified_html= htmlrag.build_block_tree(simplified_html, max_node_words=512)
clean_texts = [
    re.sub(r'<[^>]+>', '', str(node[0]))
    for node in block_tree
]

In [68]:
clean_texts

['基盤モデルを使用した生成系 AI アプリケーションの構築 – Amazon Bedrock に関するよくある質問 – AWS',
 ' メインコンテンツに移動',
 '\n\n\nAmazon Web Services のホームページに戻るには、ここをクリック\n\nAWS について\nお問い合わせ\n サポート \xa0 \n        \n 日本語 \xa0 \n        \n アカウント \xa0 \n        \n\n\n サインイン\n AWS アカウントを作成\n\n\n\n閉じる\n\n            プロファイル \n          \n\n            プロファイルは、特定の AWS エクスペリエンスとのインタラクションの改善に役立ちます。 \n          \n ログイン\n\n\n閉じる\n\n            プロファイル \n          \n\n            プロファイルは、特定の AWS エクスペリエンスとのインタラクションの改善に役立ちます。 \n          \n\n プロファイルを表示\n ログアウト\n\n\n\n\n\n\nAmazon Q\n製品\nソリューション\n料金\nドキュメント\n学ぶ\nパートナーネットワーク\nAWS Marketplace\nカスタマーサポート\nイベント\nその他 \n\n\n\n閉じる\n\n\nعربي\nBahasa Indonesia\nDeutsch\nEnglish\nEspañol\nFrançais\nItaliano\nPortuguês\n\n\nTiếng Việt\nTürkçe\nΡусский\nไทย\n日本語\n한국어\n中文 (简体)\n中文 (繁體)\n\n\n\n\n閉じる\n\n自分のプロフィール\nAWS Builder ID からサインアウト\nAWS マネジメントコンソール\nアカウント設定\n請求情報とコスト管理\nセキュリティ認証情報\nAWS Personal Health Dashboard\n\n\n\n閉じる\n\nサポートセンター\nエキスパートによるサポート\nナレッジセンター\nAWS サポートの概要\nAWS re:Post\n\n\n\n\n\nアマ